## Importando Librerias

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from imblearn.over_sampling import RandomOverSampler

## Definir el dispositivo de ejecucion (GPU si está disponible)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Directorios de las imágenes

In [3]:
train_violent_dir = "../resources/dataset/train/violencia/"
train_non_violent_dir = "../resources/dataset/train/no_violencia/"
test_violent_dir = "../resources/dataset/test/violencia/"
test_non_violent_dir = "../resources/dataset/test/no_violencia/"

## Función para cargar imágenes y etiquetas en arrays NumPy

In [4]:
def load_images_and_labels(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path).convert('RGB')
            img = img.resize((224, 224))  # Redimensionar imágenes a 224x224
            img_array = np.array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

## Cargar datos de entrenamiento

In [5]:
X_train_violent, y_train_violent = load_images_and_labels(train_violent_dir, 1)
X_train_non_violent, y_train_non_violent = load_images_and_labels(train_non_violent_dir, 0)

X_train = np.concatenate((X_train_violent, X_train_non_violent), axis=0)
y_train = np.concatenate((y_train_violent, y_train_non_violent), axis=0)
## liberando memoria
del X_train_violent
del y_train_violent
del X_train_non_violent
del y_train_non_violent 

## Aplicar sobremuestreo (oversampling) SOLO a los datos de entrenamiento

In [6]:

#ros = RandomOverSampler(random_state=42)
#X_resampled, y_resampled = ros.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)
#X_resampled = X_resampled.reshape(-1, 224, 224, 3)  # Reajustar la forma
X_resampled=X_train
y_resampled=y_train
##liberando memoria
del X_train
del y_train

## Dividir los datos de entrenamiento en entrenamiento y validación

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
##liberando memoria
del X_resampled
del y_resampled

## Convertir a tensores

In [8]:
train_inputs = torch.tensor(X_train, dtype=torch.float32).permute(0, 3, 1, 2)
train_labels = torch.tensor(y_train, dtype=torch.long)
val_inputs = torch.tensor(X_val, dtype=torch.float32).permute(0, 3, 1, 2)
val_labels = torch.tensor(y_val, dtype=torch.long).to(device)  # Mover val_labels a la GPU

## vaciando variables para desocupar RAM
del X_train
del X_val
del y_train
del y_val 


## Crear datasets y dataloaders

In [9]:

train_dataset = TensorDataset(train_inputs, train_labels)
val_dataset = TensorDataset(val_inputs, val_labels)
train_loader = DataLoader(train_dataset, batch_size=192, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=192, shuffle=True)  # DataLoader para validación
## liberando memoria


del train_inputs 
del train_labels 
del val_inputs 
del val_labels 

## Modelo

### Crear el modelo

In [10]:
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)

### Congelar capas (opcional, si no quieres entrenarlas)

In [11]:
for param in model.features.parameters():
    param.requires_grad = False

### Mover el modelo al dispositivo (CPU o GPU)

In [12]:
model = model.to(device) 

### Definir criterio de pérdida y optimizador

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Entrenamiento

In [14]:
num_train_samples = len(train_dataset)
train_losses, train_accs = [], []
val_losses, val_accs = [], []

In [15]:
epocas=4
for epoch in range(epocas):  
    model.train()  
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:  
        inputs = inputs.to(device) 
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total  # Calcular la precisión de entrenamiento
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validación (en cada época)
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:  # Usar val_loader para validación
            inputs = inputs.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) 

            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total 
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f'Epoch [{epoch+1}/{epocas}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')


#liberando memoria
torch.cuda.empty_cache()
del train_dataset
del val_dataset 
del val_loader 

Epoch [1/4], Train Loss: 1.6222, Train Acc: 87.46%, Val Loss: 0.1150, Val Acc: 94.96%
Epoch [2/4], Train Loss: 0.1146, Train Acc: 95.68%, Val Loss: 0.0288, Val Acc: 99.31%
Epoch [3/4], Train Loss: 0.0574, Train Acc: 98.20%, Val Loss: 0.0124, Val Acc: 99.57%
Epoch [4/4], Train Loss: 0.0356, Train Acc: 98.98%, Val Loss: 0.0102, Val Acc: 99.70%


## Guardar el modelo

In [16]:

torch.save(model.state_dict(), "../modelos/VGG16_pytorch.pth")
del optimizer
del train_loader


del num_train_samples
del train_losses
del  train_accs 
del val_losses
del val_accs 

## Test

### Cargar datos de prueba 

In [17]:

X_test_violent, y_test_violent = load_images_and_labels(test_violent_dir, 1)
X_test_non_violent, y_test_non_violent = load_images_and_labels(test_non_violent_dir, 0)

X_test = np.concatenate((X_test_violent, X_test_non_violent), axis=0)
y_test = np.concatenate((y_test_violent, y_test_non_violent), axis=0)

##liberando memoria
del X_test_violent
del y_test_violent
del X_test_non_violent
del y_test_non_violent 


### Convertir a tensores

In [18]:
test_inputs = torch.tensor(np.array(X_test), dtype=torch.float32).permute(0, 3, 1, 2)
test_labels = torch.tensor(y_test, dtype=torch.long)

### Crear dataset de prueba

In [19]:
test_dataset = TensorDataset(test_inputs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
###liberando memoria
del test_inputs 
del test_labels 
del X_test 
del y_test 


## Evaluación en el conjunto de prueba 

In [20]:

model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:  
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

###  Métricas para conjuntos de datos desbalanceados

In [21]:

conf_matrix = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
gmean = np.sqrt(recall * (tn / (tn + fp)))  # G-Mean

print(f"Test Accuracy: {100 * np.mean(np.array(y_true) == np.array(y_pred)):.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")


Test Accuracy: 93.33%
Precision: 0.9447
Recall: 0.6153
F1 Score: 0.7452
G-Mean: 0.7817


### Visualización de la matriz de confusión

In [22]:

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["No Violencia", "Violencia"], yticklabels=["No Violencia", "Violencia"])
plt.xlabel("Clase Predicha")
plt.ylabel("Clase Real")
plt.title("Matriz de Confusión")
plt.show()




In [23]:
print(conf_matrix)

[[6289   43]
 [ 459  734]]
